In [75]:
from CoolProp import CoolProp
import numpy as np

print(help(CoolProp))
#fls = CoolProp.FluidsList
# abste = CoolProp.AbstractState
# print(help(abste))
#print(abste())
#print(fls())
#print(help(fls))

Help on module CoolProp.CoolProp in CoolProp:

NAME
    CoolProp.CoolProp

CLASSES
    builtins.object
        AbstractState
        PyCriticalState
        PyGuessesStructure
        PyPhaseEnvelopeData
        PySpinodalData
        State
    
    class AbstractState(builtins.object)
     |  This class is a one-to-one python wrapper of the :cpapi:`AbstractState` class
     |  
     |  Methods defined here:
     |  
     |  Bvirial(...)
     |      AbstractState.Bvirial(self) -> double
     |      Get the B virial coefficient - wrapper of c++ function :cpapi:`CoolProp::AbstractState::Bvirial(void)`
     |  
     |  Cvirial(...)
     |      AbstractState.Cvirial(self) -> double
     |      Get the C virial coefficient - wrapper of c++ function :cpapi:`CoolProp::AbstractState::Cvirial(void)`
     |  
     |  PIP(...)
     |      AbstractState.PIP(self) -> double
     |      Get the phase identification parameter - wrapper of c++ function :cpapi:`CoolProp::AbstractState::PIP`
     |  
  

# Motor do ciclo Otto

Para o motor do ciclo Otto, temos a Figura


As suas características, de um ciclo ideal, são:

* De **A** pra **B** temos compressão isoentrópica $s=\text{const}$
* De **B** pra **C** temos um aquecimento isovolumétrico $v=\text{const}$
* De **C** pra **D** temos uma expansão isoentrópica $s=\text{const}$
* De **D** pra **A** temos um resfriamento isovolumétrico $v=\text{const}$

Como configuração do projeto temos a razão de compressão dada por

$$
CR = \dfrac{V_A}{V_B} \in \left[5, \ 20\right]
$$

e o fluido que poderemos utilizar:

* Metano: ```fluido = "methane"```
* Etanol: ```fluido = "ethanol"```
* Gasolina: ```fluido = "nOctane"```

In [50]:
fluido = "methane"
fluido = "ethanol"
fluido = "nOctane"

Agora colocamos as condições de contorno: Volume máximo/volume mínimo, pressão máxima e pressão minima.

In [51]:
CR = 10  # exemplo
AF = 9  # air ratio
Ncyl = 4  # numero de cilindros
N = 3600  # rpm
N /= 60  # transforma para segundos
HH = 44e+6  # J/kg, para a gasolina
Tamb = 32 + 273.15  # Temperatura ambiente de 32 graus
Pamb = 100e+3  # Pa
Pmax = 600000  # Pa

bore = 0.100  # m, diâmetro do pistão
stroke = 0.110  # m, deslocamento do pistão

$$
CR = \dfrac{V_A}{V_B} = \dfrac{\pi r^2 \cdot L_{max}}{\pi r^2 \cdot L_{min}} = \dfrac{L_{max}}{L_{min}}
$$
$$
\Delta L = L_{max} - L_{min} \Rightarrow L_{max} = L_{min} + \Delta L
$$
$$
CR \cdot L_{min} = L_{min} + \Delta L \Rightarrow L_{min} = \dfrac{\Delta L}{CR-1}
$$

In [55]:
Lmin = stroke/(CR-1)
Lmax = Lmin + stroke
Vmin = np.pi * bore**2 * Lmin/4
Vmax = np.pi * bore**2 * Lmax/4
print("Lmin = %.3f m" % Lmin)
print("Lmax = %.3f m" % Lmax)
print("Vmin = %.3f L" % (1000*Vmin))
print("Vmax = %.3f L" % (1000*Vmax))

Lmin = 0.012 mm
Lmax = 0.122 mm
Vmin = 0.096 L
Vmax = 0.960 L


In [58]:
T1 = Tamb
P1 = Pamb
s1 = CoolProp.PropsSI("S", "T", T1, "P", P1, "air")
# h1 = CoolProp.PropsSI("H", "T", T1, "P", P1, fluido)
u1 = CoolProp.PropsSI("U", "T", T1, "P", P1, "air")
v1 = CoolProp.PropsSI("V", "T", T1, "P", P1, "air")
V1 = Vmax
m1 = V1/v1
print("T1 = %.3f oC" % (T1-273.15))
print("P1 = %.3f kPa" % (P1/1000))
print("s1 = %.3f kJ/kg*K" % (s1/1000))
print("u1 = %.3f kJ/kg" % (u1/1000))
print("V1 = %.3f L" % (1000*V1))
print("v1 = %.3f L/kg" % (1000*v1))
print("m1 = %.3f kg/s" % (m1))

T1 = 32.000 oC
P1 = 100.000 kPa
s1 = 3.908 kJ/kg*K
u1 = 343.914 kJ/kg
V1 = 0.960 L
v1 = 0.019 L/kg
m1 = 51.103 kg/s


In [61]:
m2 = m1
V2 = Vmin
v2 = V2/m2
s2 = s1
print("m2 = %.3f kg/s" % (m2))
print("V2 = %.3f L" % (1000*V2))
print("v2 = %.3f L/kg" % (1000*v2))
print("s2 = %.3f kJ/kg*K" % (s2/1000))

P2 = CoolProp.PropsSI("P", "V", v2, "S", s2, "air")
T2 = CoolProp.PropsSI("T", "V", v2, "S", s2, "air")
# h2 = CoolProp.PropsSI("H", "V", v2, "S", s2, fluido)
u2 = CoolProp.PropsSI("U", "V", v2, "S", s2, "air")
print("T2 = %.3f oC" % (T2-273.15))
print("P2 = %.3f kPa" % (P2/1000))
print("u2 = %.3f kJ/kg" % (u2/1000))

m2 = 51.103 kg/s
V2 = 0.096 L
v2 = 0.002 L/kg
s2 = 3.908 kJ/kg*K


ValueError: Input pair variable is invalid and output(s) are non-trivial; cannot do state update : PropsSI("P","V",1.878441838e-06,"S",3907.632174,"air")